In [ ]:
import os
import pandas as pd

main_folder = "data"

article_ids = []
langs = []
texts = []

for lang_folder in os.listdir(main_folder):
    lang_path = os.path.join(main_folder, lang_folder)
    if os.path.isdir(lang_path):
        lang_code = lang_folder
        # Iterate over files in train-articles-subtask-3 folder
        train_folder = os.path.join(lang_path, "train-articles-subtask-3")
        for file in os.listdir(train_folder):
            if file.endswith(".txt"):
                file_path = os.path.join(train_folder, file)
                # Extract ID from file name
                article_id = file.split(".")[0].split("article")[-1]
                # Read text from file
                try:
                    with open(file_path, "r") as f:
                        text = f.read()
                    article_ids.append(article_id)
                    langs.append(lang_code)
                    texts.append(text)
                except UnicodeDecodeError:
                    pass

# Create pandas dataframe
df = pd.DataFrame({
    "article_id": article_ids,
    "lang": langs,
    "text": texts
})

# Display dataframe
print(df)

df.to_csv('train.tsv', sep='\t')

In [ ]:
import json

# Load articles from JSON file
articles = json.load(open('train.json', 'r', encoding='utf-8'))

# Load gold standard annotations
with open('train-labels-subtask-3-spans.txt', 'r', encoding='utf-8') as f:
    gold_standard = [line.strip().split('\t') for line in f]

# Create a dictionary to store annotations by article ID
annotations_by_article = {article['article_id']:[] for article in articles}

# Populate annotations dictionary with gold standard annotations
for annotation in gold_standard:
    article_id, tag, start, end = annotation
    annotations_by_article[article_id].append({
        'start': int(start),
        'end': int(end),
        'tag': tag  # The alignment system expects 'tag'
    })

# Add annotations to articles in the JSON file
for article in articles:
    article_id = article['article_id']
    if article_id in annotations_by_article:
        article['annotations'] = annotations_by_article[article_id]
    else:
        article['annotations'] = []

# Save modified articles back to JSON file
with open('annotated_train.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [ ]:
import pandas as pd

df = pd.read_json('annotated_train.json')
df